<a href="https://colab.research.google.com/github/sksatyam17/chatbot-with-sentiment-analysis/blob/main/chadbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import torch

In [3]:
login(token="your token here") #kindly use your own token. thank you.

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# Define the path for the CSV file
csv_file = 'qa_dataset.csv'

# Check if the CSV file exists; if not, create it with initial data
if not os.path.exists(csv_file):
    qa_data = {
        'question': ["What is the name of Julius Magellan's dog?", "Who is Julius Magellan's dog?"],
        'answer': ["The name of Julius Magellan's dog is Sparky", "Julius Magellan's dog is called Sparky"]
    }
    qa_df = pd.DataFrame(qa_data)
    qa_df.to_csv(csv_file, index=False)
else:
    # Load the existing CSV file into a DataFrame
    qa_df = pd.read_csv(csv_file)

In [5]:
# Verify the CSV content
qa_df = pd.read_csv(csv_file)
print(qa_df)

                                     question  \
0  What is the name of Julius Magellan's dog?   
1               Who is Julius Magellan's dog?   

                                        answer  
0  The name of Julius Magellan's dog is Sparky  
1       Julius Magellan's dog is called Sparky  


In [6]:
# Initialize the Llama 2 model and tokenizer
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

# Initialize the pipeline using Hugging Face pipeline
llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1024,  # Adjust max_length as needed
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [40]:
sentiment_analyzer = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [41]:
# Test the sentiment analyzer on an example sentence
example_sentence = "I can't live without her. Give me some tips."
sentiment_analysis = sentiment_analyzer(example_sentence)

print(sentiment_analysis)


[{'label': 'POSITIVE', 'score': 0.9978491067886353}]


In [33]:
def modified_question_prompt(question):
    return f"As a customer support agent, provide helpful tips in response to this inquiry: '{question}'"


In [42]:
def answer_question(question):
    global qa_df
    # Analyze sentiment of the user's input
    sentiment = sentiment_analyzer(question)[0]['label']

    # Modify the input question to guide Llama 2 to respond appropriately
    modified_question = modified_question_prompt(question)

    # Check if the question is in the QA dataset
    answer = qa_df[qa_df['question'].str.lower() == question.lower()]['answer']

    if not answer.empty:
        response = answer.iloc[0]
        print(f"Answer from QA dataset: {response}")
    else:
        # Generate response using Llama 2 with the modified prompt
        response = llama_pipeline(modified_question, max_length=150, do_sample=True)[0]['generated_text']

        # Clean the response and remove redundant parts
        response = response.replace(f"Answer: {modified_question}", "").strip()

        # Adjust the response based on sentiment
        if sentiment == 'NEGATIVE':
            response = f"I'm sorry you're feeling this way. {response}"
        elif sentiment == 'POSITIVE':
            response = f"I'm glad you're doing well! {response}"

        print(f"Answer from Llama 2: {response}")

        # Add the new QA pair to the dataset if it's not already present
        if not any(qa_df['question'].str.lower() == question.lower()):
            new_row = pd.DataFrame({'question': [question], 'answer': [response]})
            qa_df = pd.concat([qa_df, new_row], ignore_index=True)
            qa_df.to_csv(csv_file, index=False)
            print("New QA pair added to the dataset.")

    return response


In [43]:
question_1 = "I left my wife alone for few days, I can't live without her. Give me some tips"
answer_question(question_1)

Answer from QA dataset: I left my wife alone for few days, I can't live without her. Give me some tips to overcome this feeling.
You're not alone in feeling this way! It's common for people to feel anxious or upset when they're away from their partners, especially if they're in a long-term relationship. Here are some tips that may help you overcome these feelings:

1. Communicate with your partner: Reach out to your wife and talk to her about how you're feeling. Share your feelings with her and ask her to reassure you that everything is okay. Hearing her voice and knowing that she's safe and happy can help ease your anxiety.


"I left my wife alone for few days, I can't live without her. Give me some tips to overcome this feeling.\nYou're not alone in feeling this way! It's common for people to feel anxious or upset when they're away from their partners, especially if they're in a long-term relationship. Here are some tips that may help you overcome these feelings:\n\n1. Communicate with your partner: Reach out to your wife and talk to her about how you're feeling. Share your feelings with her and ask her to reassure you that everything is okay. Hearing her voice and knowing that she's safe and happy can help ease your anxiety."

In [10]:
question_2 = "Who is Julius Magellan's dog?"
answer_question(question_2)

Answer from QA dataset: Julius Magellan's dog is called Sparky


"Julius Magellan's dog is called Sparky"

In [11]:
# This should fallback to Llama 2 and then get added
# to the dataset
question_3 = "What is the capital of France?"
answer_question(question_3)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Answer from Llama 2: That's great to hear! What is the capital of France?

The capital of France is Paris.
New QA pair added to the dataset.


"That's great to hear! What is the capital of France?\n\nThe capital of France is Paris."

In [12]:
print(qa_df)

                                     question  \
0  What is the name of Julius Magellan's dog?   
1               Who is Julius Magellan's dog?   
2              What is the capital of France?   

                                              answer  
0        The name of Julius Magellan's dog is Sparky  
1             Julius Magellan's dog is called Sparky  
2  That's great to hear! What is the capital of F...  


In [13]:
!pip -q install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.6 MB/s eta 0:00:00


In [26]:
import gradio as gr

def gradio_chat_interface(question):
    global qa_df
    answer = qa_df[qa_df['question'].str.lower() == question.lower()]['answer']

    if not answer.empty:
        return f"Answer from QA dataset: {answer.iloc[0]}"
    else:
        response = llama_pipeline(question, max_length=150, do_sample=True)[0]['generated_text']
        response = response.replace(f"Answer: {question}", "").strip()
        # Add new question-answer pair to the dataset
        if not any(qa_df['question'].str.lower() == question.lower()):
            new_row = pd.DataFrame({'question': [question], 'answer': [response]})
            qa_df = pd.concat([qa_df, new_row], ignore_index=True)
            qa_df.to_csv(csv_file, index=False)
            return f"Answer from Llama 2: {response} \n(New QA pair added to the dataset.)"

In [36]:
# Create a Gradio Interface
interface = gr.Interface(
    fn=gradio_chat_interface,
    inputs="text",
    outputs="text",
    title="Llama 2 Chatbot with QA Pipeline, sentiment analysis",
    description="Ask a question and the chatbot will respond using a pre-defined QA dataset or Llama 2 if the answer is not in the dataset.",
)

In [44]:
# Launch the Gradio Interface
interface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0bcd3a1ff393f9ec25.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
